# CS 236 Project Phase 1: Data Cleaning and EDA

Installation

```
pip install pyspark
```

We couldn't get PySpark 3 to work on our machines, so we used the latest PySpark 4 instead.
We had some issues with the Java version - Java 25 is not supported, so we used Java 21.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

hotel_booking = spark.read.csv("datasets/hotel-booking.csv", header=True)
customer_reservations = spark.read.csv(
    "datasets/customer-reservations.csv", header=True
)

print(hotel_booking)
print(customer_reservations)

DataFrame[hotel: string, booking_status: string, lead_time: string, arrival_year: string, arrival_month: string, arrival_date_week_number: string, arrival_date_day_of_month: string, stays_in_weekend_nights: string, stays_in_week_nights: string, market_segment_type: string, country: string, avg_price_per_room: string, email: string]
DataFrame[Booking_ID: string, stays_in_weekend_nights: string, stays_in_week_nights: string, lead_time: string, arrival_year: string, arrival_month: string, arrival_date: string, market_segment_type: string, avg_price_per_room: string, booking_status: string]


There are two parts that need to be aligned. We need to rename `arrival_date` to match `arrival_date_day_of_month` and map the booking status from Canceled and Not_Cancelled to 0 and 1.

In [5]:
customer_reservations = customer_reservations.withColumnRenamed("arrival_date", "arrival_date_day_of_month")

After we align the two datasets, we can merge them and write to a new file.

In [6]:
# merged_df = hotel_booking.union(customer_reservations)
# with open('merged.csv') as f:
#     merged_df.write.csv(f, mode='overwrite')